In [1]:
import requests
import time
import json
import numpy as np
import blockchain

In [2]:
url = "https://api.thingspeak.com/channels/1704499/feeds.json?api_key=Z70IVJ1X27AVPQ6C&results=2"
Rkey = "MC9FFB16PZV2BSWR"
R = 6373.0 # world radius
latStation = [7.865661,7.881308,7.893300,7.887864,7.890755,7.882510,7.885414] # lat of stantion point
lonStation = [98.397750,98.364982,98.368747,98.371470,98.390563,98.404646,98.430988] # lon of stantion point

In [3]:
def readSpeed(time) : #calculate distance from lat and lot then find the speed by v = s/t
    

    resp = requests.get(url) #request url
    data_disc = json.loads(resp.text) #converst to json format
    #convest data to radians value for process in the next step
    lat1 = np.radians(float(data_disc["feeds"][0]["field1"])) 
    lon1 = np.radians(float(data_disc["feeds"][0]["field2"]))
    
    lat2 = np.radians(float(data_disc["feeds"][1]["field1"]))
    lon2 = np.radians(float(data_disc["feeds"][1]["field2"]))
    #find difference between point 1 and point 2
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    speed = (distance*3600)/time; #km/hr

    return speed


In [4]:
def distanceDriver_station(latSt,lonSt):#calculate distance from location of driver and station
    
    resp = requests.get(url)
    data_disc = json.loads(resp.text)
    
    lat1 = np.radians(float(data_disc["feeds"][1]["field1"]))
    lon1 = np.radians(float(data_disc["feeds"][1]["field2"]))
    
    lat2 = np.radians(latSt)
    lon2 = np.radians(lonSt)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    
    return distance

In [5]:
def addData(dt1,dt2,dt3,dt4,dt5,dt6,dt7,dt8): # add the data into thing speak
    urlWrite = "https://api.thingspeak.com/update?api_key="+ Rkey + "&field1=" + str(dt1) + "&field2=" +  str(dt2) + "&field3=" +  str(dt3) + "&field4=" +  str(dt4) + "&field5=" +  str(dt5) + "&field6=" +  str(dt6) + "&field7=" +  str(dt7) + "&field8=" +  str(dt8)
    requests.get(urlWrite)

In [6]:
def getDis() :

    resp = requests.get(url)
    data_disc = json.loads(resp.text)

    stationId = data_disc["feeds"][1]["field4"]

    if stationId == "1.0" :
        dis = distanceDriver_station(latStation[0],lonStation[0])  
    
    elif stationId == "2.0" :
        dis = distanceDriver_station(latStation[1],lonStation[1])
    
    elif stationId == "3.0" :
        dis = distanceDriver_station(latStation[2],lonStation[2])
    
    elif stationId == "4.0" :
        dis = distanceDriver_station(latStation[3],lonStation[3])
    
    elif stationId == "5.0" :
        dis = distanceDriver_station(latStation[4],lonStation[4])
    
    elif stationId == "6.0" :
        dis = distanceDriver_station(latStation[5],lonStation[5])
    
    elif stationId == "7.0" :
        dis = distanceDriver_station(latStation[6],lonStation[6])
      
    return dis


In [7]:
bc = blockchain.Blockchain()# create first blockchain

In [8]:
while (bc.is_chain_valid()):# check validation of blockchain
    resp = requests.get(url)
    data_disc = json.loads(resp.text)

    stationId = data_disc["feeds"][1]["field4"]
    
    distance = getDis()
    speed = readSpeed(30) # calculate the speed when t  = 30s because driver divec will be update every 30 s
    estimate = (distance/speed)* 60 # for min
    
    bc.mine_block("Station :" + str(stationId) + ",Distance :"+ str(distance) + ",Speed :" + str(speed) + ",Estimate time :" + str(estimate))
    #create the data by use mine_blockmethod
    addData(stationId,distance,speed,estimate,bc.is_chain_valid(),bc.chain[len(bc.chain)-1]["index"],bc.chain[len(bc.chain)-1]["proof"],bc.chain[len(bc.chain)-1]["previous_hash"])
    print(bc.chain[len(bc.chain)-1])# print the current block
    time.sleep(30)

C:\Users\ziver\AppData\Local\Temp\ipykernel_12044\2068764997.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  estimate = (distance/speed)* 60 # for min


{'index': 2, 'timestamp': '2022-10-09 21:15:03.653436', 'data': 'Station :3.0,Distance :6.317749009244943,Speed :0.0,Estimate time :inf', 'proof': 2731, 'previous_hash': '6acbacd64ea47c316a3648b47c5b187c4ccc60c56c70be6451813afe964dce62'}
{'index': 3, 'timestamp': '2022-10-09 21:15:39.565563', 'data': 'Station :3.0,Distance :6.317749009244943,Speed :0.0,Estimate time :inf', 'proof': 1351, 'previous_hash': 'bb37f95e8be82bb7a9bef425de03bbe998e5dd4a0a53c6747396de1e9b4fa986'}
{'index': 4, 'timestamp': '2022-10-09 21:16:15.640121', 'data': 'Station :3.0,Distance :6.317749009244943,Speed :0.0,Estimate time :inf', 'proof': 3969, 'previous_hash': '8f6d82ee306c3b9157326c450b4858e0833d139455c40d21b168d1339cc6a669'}
{'index': 5, 'timestamp': '2022-10-09 21:16:51.545292', 'data': 'Station :1.0,Distance :8.552194204467092,Speed :0.11848815424087317,Estimate time :4330.657824451263', 'proof': 5913, 'previous_hash': '695da8a2b30e22f11c2f687aa439bed9543707b485efd5bc95efd2dcf5a4abff'}
{'index': 6, 'time

KeyboardInterrupt: 